In [ ]:
!nvidia-smi

Sat Apr 23 20:48:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
 
drive.mount('/content/drive', force_remount=True)
COLAB = True
print("Note: using Google CoLab")

Mounted at /content/drive
Note: using Google CoLab


In [ ]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=5d8077f31b3eaa7faa47438e11fc7c6e1e2d87dc90f05c92974f3727c5d64a9c
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [ ]:
%cd drive/MyDrive/Makeathon_TUM/CoVision/classification_model_training/

/content/drive/MyDrive/Makeathon_TUM/CoVision/classification_model_training


In [ ]:
%ls

helper_funcs.ipynb  Output_path/  predict.py  train.py  utils/


# Training

In [ ]:
PATH_IMG = "/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/train/images"
PATH_GT = "/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/train/train.csv"
PATH_EXP = "/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/03"

!python train.py --num_classes 3 --pretrained_on_ImageNet --fold 1 --dataset {PATH_IMG} --gt {PATH_GT} --outdir {PATH_EXP}  --epochs 30

# Testing

In [ ]:
PATH_TEST_IMG = "/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/test/images"
PATH_TEST_GT = "/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/test/test.csv"
MODEL = "/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/02/model_fold_1_20.pt"


!python predict.py --num_classes 3 --single_model_path {MODEL} --dataset {PATH_TEST_IMG} --gt {PATH_TEST_GT} 

# Prediction on single image

In [ ]:
import torch
import cv2
from torch.utils.data import Dataset, DataLoader

import numpy as np 

from PIL import Image
from PIL import ImageFile

In [ ]:
from efficientnet_pytorch import EfficientNet
import torch
import numpy as np
from torchvision import transforms
from PIL import Image
from torch.autograd import Variable

model_path = r"/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_classification/experiments/01/model_fold_1_5.bin"

model = EfficientNet.from_name("efficientnet-b2", in_channels = 3, num_classes = 1)
model.load_state_dict(torch.load(model_path))
model.to("cpu")

In [ ]:
image_path_1 = r"/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_classification/cropped/test/images/test_negative_8.jpg"
image_path_2 = r"/content/drive/MyDrive/Makeathon_TUM/data/rapid_test_classification/cropped/test/images/test_positive_9.jpg"

model.eval()

loader = transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()])

def image_loader(image_name):
    """load image, returns cuda tensor"""
    image = Image.open(image_name)
    image = loader(image).float()
    image = Variable(image, requires_grad=True)
    image = image.unsqueeze(0)  #this is for VGG, may not be needed for ResNet
    # return image.cuda()  #assumes that you're using GPU
    return image

image = image_loader(image_path_2)

output = model(image)
output = torch.sigmoid(output)

print()
print("output: ", output)
print()



output:  tensor([[0.9997]], grad_fn=<SigmoidBackward0>)



# Convert to Onnx and than to tf.js

## Convert .bin to onnx

In [ ]:
import torch 
from efficientnet_pytorch import EfficientNet
from torch.autograd import Variable
import os

model = EfficientNet.from_name("efficientnet-b2", in_channels = 3, num_classes = 3)
model.load_state_dict(torch.load(MODEL))
model.set_swish(memory_efficient=False)

dummy_input = Variable(torch.rand(1, 3, 224, 224))
torch.onnx.export(model, dummy_input, os.path.join(PATH_EXP, "best_model.onnx"))

## Convert .onnx to tensorflow saved model

In [ ]:
!pip install onnx-tf

In [ ]:
INPUT_MODEL = os.path.join(PATH_EXP, "best_model.onnx")
OUTPUT_MODEL = os.path.join(PATH_EXP, "best_model.pb")


import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load(INPUT_MODEL)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph(OUTPUT_MODEL)  # export the model


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/03/best_model.pb/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/03/best_model.pb/assets


## Tensorflow saved model to tf.js

In [ ]:
!pip install tensorflowjs

In [ ]:
!tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names=tfjs_layers_model \
    --saved_model_tags=serve \
    /content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/03/best_model.pb \
    /content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/03/

2022-04-24 00:39:43.565063: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Writing weight file /content/drive/MyDrive/Makeathon_TUM/data/rapid_test_three_class_classification/experiments/03/model.json...
